In [1]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using CSV
using StatsBase
include("utils.jl")
include("financial_bvar_temp.jl")
using .FinancialBVAR

In [2]:
data = TimeArray(CSV.File("usa_data_transformed.csv"),timestamp=:timestamp)
start = Date(1993,03,01)
end_d = Date(2023, 12, 31)
s_data = to(from(data, start),end_d)

124×6 TimeArray{Float64, 2, Date, Matrix{Float64}} 1993-03-01 to 2023-12-01
┌────────────┬─────────────────┬───────────────┬─────────────┬───────────────┬──
│            │ Total_Stock_CPI │ Price_10Y_CPI │ T_Bill_CPI  │ Dividend_Yeld │ ⋯
├────────────┼─────────────────┼───────────────┼─────────────┼───────────────┼──
│ 1993-03-01 │       0.0150658 │     0.0543577 │ -0.00250148 │     0.0281965 │ ⋯
│ 1993-06-01 │       0.0136547 │     0.0292271 │  0.00478562 │     0.0279908 │ ⋯
│ 1993-09-01 │       0.0342312 │       0.03506 │ -0.00169417 │     0.0270317 │ ⋯
│ 1993-12-01 │       0.0226641 │    -0.0219636 │  0.00410946 │     0.0266883 │ ⋯
│ 1994-03-01 │      -0.0571526 │    -0.0622961 │ 0.000425924 │     0.0285162 │ ⋯
│ 1994-06-01 │      0.00958316 │    -0.0293652 │  0.00338014 │     0.0285113 │ ⋯
│ 1994-09-01 │        0.026626 │   -0.00884948 │  0.00390608 │     0.0280574 │ ⋯
│ 1994-12-01 │      0.00488791 │  -0.000941161 │  0.00828514 │     0.0283289 │ ⋯
│     ⋮      │        ⋮        │ 

In [3]:
vmodel = VARModel(s_data)
scenarios = simulate(vmodel,60, 100_000)
returns_scenarios = scenarios[1:3,:,:]
#dodac exponential

3×100000×61 Array{Float64, 3}:
[:, :, 1] =
 0.121595   0.121595   0.121595   …  0.121595   0.121595   0.121595
 0.0651554  0.0651554  0.0651554     0.0651554  0.0651554  0.0651554
 0.0103466  0.0103466  0.0103466     0.0103466  0.0103466  0.0103466

[:, :, 2] =
  0.0842427    0.122827      0.0644005   …  0.0023326   0.0952391   0.0832954
 -0.0313281   -0.0134458     0.0189846      0.0720547  -0.0504327   0.0125433
  0.00172332  -0.000838539  -0.00241014     0.0144822   0.00117056  0.0105001

[:, :, 3] =
 0.0997077    0.0528716    -0.0991274   …  0.0440792   0.052793    0.0179908
 0.00296932  -0.00560649    0.0326503      0.0628554   0.0320883   0.0162121
 0.0115345   -0.000587626   0.00713588     0.00158695  0.00890991  0.00832955

;;; … 

[:, :, 59] =
  0.0126885   0.053589      0.121593    …  -0.0393985  -0.123989   -0.10493
 -0.0377678  -0.0538786    -0.00562993      0.0751866  -0.0176268  -0.0132866
 -0.0095373   0.000382017   0.00197949      0.0111714  -0.0021734  -0.015099

[:, :

In [19]:
includet("kmeans_moments_lattice.jl")
using .LatticeGeneration

In [20]:
returns_scenarios

3×100000×61 Array{Float64, 3}:
[:, :, 1] =
 0.121595   0.121595   0.121595   …  0.121595   0.121595   0.121595
 0.0651554  0.0651554  0.0651554     0.0651554  0.0651554  0.0651554
 0.0103466  0.0103466  0.0103466     0.0103466  0.0103466  0.0103466

[:, :, 2] =
  0.0842427    0.122827      0.0644005   …  0.0023326   0.0952391   0.0832954
 -0.0313281   -0.0134458     0.0189846      0.0720547  -0.0504327   0.0125433
  0.00172332  -0.000838539  -0.00241014     0.0144822   0.00117056  0.0105001

[:, :, 3] =
 0.0997077    0.0528716    -0.0991274   …  0.0440792   0.052793    0.0179908
 0.00296932  -0.00560649    0.0326503      0.0628554   0.0320883   0.0162121
 0.0115345   -0.000587626   0.00713588     0.00158695  0.00890991  0.00832955

;;; … 

[:, :, 59] =
  0.0126885   0.053589      0.121593    …  -0.0393985  -0.123989   -0.10493
 -0.0377678  -0.0538786    -0.00562993      0.0751866  -0.0176268  -0.0132866
 -0.0095373   0.000382017   0.00197949      0.0111714  -0.0021734  -0.015099

[:, :

In [21]:
s = sum(returns_scenarios[:,:,21:25]; dims=3)
s = dropdims(s; dims= 3)
mean(s[1,:,:])

0.06411551431523302

In [22]:
returns_scenarios_ann = zeros(3,100_000,15)
for asset in 1:3
    returns_scenarios_ann[asset,:,:] = annualise(returns_scenarios[asset,:,2:end],4)
end
returns_scenarios_ann


3×100000×15 Array{Float64, 3}:
[:, :, 1] =
 0.358968    0.319918    -0.0228422  …  -0.0672574    0.0406951   0.245601
 0.0372686  -0.0387406    0.0964813      0.112496    -0.0824731  -0.0276212
 0.0317517  -0.00049309   0.0163916      0.00457973   0.0064461   0.0135794

[:, :, 2] =
 0.0730724  -0.0241319  0.204063   …  0.224259    0.0422959  -0.126721
 0.0369356   0.0521555  0.0223712     0.0110647   0.209331    0.0139704
 0.013967    0.0218049  0.0392606     0.00982141  0.0303188   0.0184117

[:, :, 3] =
  0.138509     0.107442    -0.201801   …  -0.082568    0.0204148   0.05483
 -0.00819037  -0.223093     0.0467488     -0.0263024   0.0128262  -0.13378
  0.0177942    0.00904608  -0.017401       0.00599709  0.017367   -0.0183518

;;; … 

[:, :, 13] =
 0.0974643   0.182969    0.188694    …   0.051839     0.212348    0.0111515
 0.0958767  -0.0101815  -0.124426       -0.0777304   -0.0259981  -0.0480559
 0.0102012  -0.012714   -0.00508912     -0.00625449  -0.0195759  -0.0290754

[:, :, 14] 

In [23]:
quantile(returns_scenarios_ann[1,:,5],[0.1,0.5,0.9])

3-element Vector{Float64}:
 -0.1432601257406753
  0.05067766181558846
  0.24430730421181965

In [24]:
lattice = cluster(s,7)

lattice[1]

7×3 transpose(::Matrix{Float64}) with eltype Float64:
 -0.073306    0.140537    0.0303294
  0.0296592  -0.032546    0.0185446
  0.285871   -0.0673584   0.000287974
  0.0580508  -0.10216    -0.0271311
  0.101236    0.0627795  -0.00600926
 -0.155281    0.0261714  -0.00855892
  0.205397    0.0802402   0.0355357

In [25]:
s

3×100000 Matrix{Float64}:
 0.0286877  0.125454    0.0986654  …   0.288702   -0.00407632  0.0823901
 0.0144129  0.0538527   0.0105315     -0.0412967   0.103445    0.0621504
 0.0232093  0.0512085  -0.0181913     -0.0217616  -0.0147051   0.0345798

In [26]:
clusters_probs(lattice[2])

MethodError: MethodError: no method matching clusters_probs(::Vector{Int64})

Closest candidates are:
  clusters_probs(::Vector{Int64}, !Matched::Int64)
   @ Main.LatticeGeneration c:\Users\matsz\programowanie\Optymalizacja_portfela\julia_msp\kmeans_moments_lattice.jl:18


In [27]:
successors_probs([1 1 2 3 3 2 2 1], [2 1 3 2 3 3 1 1 ], 5, 2)

5-element Vector{Float64}:
 0.3333333333333333
 0.0
 0.6666666666666666
 0.0
 0.0

In [28]:
[count(x -> x == n, [2 1 3 2 3 3 1 1 ]) for n in 1:4]

4-element Vector{Int64}:
 3
 2
 3
 0

In [29]:
counts([2 1 3 2 3 3 1 1 9], 9)

9-element Vector{Int64}:
 3
 2
 3
 0
 0
 0
 0
 0
 1

In [36]:
clusters_probs([2, 1, 3, 2, 3, 3, 1, 1], 5)

5-element Vector{Float64}:
 0.375
 0.25
 0.375
 0.0
 0.0

In [71]:
res = generate_lattice(returns_scenarios_ann, 7,15)


Lattice([0.30495771126914256 -0.06017948801580175 … 0.2689749302414999 0.10346140066054149; -0.08359393604541467 0.14965936253813317 … 0.03987525532062344 0.11273145080619709; … ; 0.17089553258968565 -0.1422898076515723 … 0.022084696281011847 -0.08186375333122344; 0.08378003251886075 0.0648059242667704 … -0.1471147938464164 -0.07147518805471864;;; -0.0005345590769979015 0.023044125258716405 … -0.10849860007229722 0.08014308049536785; -0.02009729573764078 -0.10152932906368672 … -0.03797499011992557 0.05222688675775623; … ; 0.073546842138934 0.005544230618907832 … -0.038306680366974565 0.12027100553775154; 0.052674512468100655 0.11601392368510957 … 0.08170260554939121 -0.05207677917816925;;; 0.019644879959985567 0.015324250069302985 … -0.0014328741746234882 0.031066687198615252; -0.0026379772031924435 -0.007717589989132087 … 0.02174154627735755 0.0051866671656900995; … ; 0.02534277016140735 -0.0174460243626247 … 0.008279369489111017 0.019124429757033958; -0.013101379197019657 0.028652425

In [74]:
round.(res.states[6,:,:]; digits=4)

7×3 Matrix{Float64}:
 -0.0581   0.1287   0.0275
  0.0263  -0.0899  -0.0242
  0.2475  -0.0712  -0.0035
  0.0209  -0.0327   0.0149
  0.0888   0.0541  -0.0053
  0.1931   0.0618   0.0282
 -0.151    0.0329  -0.0051

In [75]:
round.(res.probs[7]; digits=2)

7×7 Matrix{Float64}:
 0.17  0.1   0.12  0.21  0.12  0.15  0.12
 0.08  0.24  0.15  0.12  0.19  0.06  0.17
 0.11  0.15  0.18  0.14  0.19  0.12  0.11
 0.15  0.16  0.09  0.16  0.16  0.09  0.19
 0.14  0.1   0.17  0.19  0.13  0.2   0.08
 0.22  0.06  0.11  0.19  0.13  0.21  0.09
 0.12  0.18  0.12  0.12  0.2   0.08  0.17